In [1]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import RobertaModel
from transformers import RobertaTokenizer
from transformers import AdamW
from sklearn.metrics import f1_score

In [2]:
# Load the CSV file
def load_data(file_path):
    df = pd.read_csv(file_path)
    texts = df["text"].tolist()
    labels = df[["anger", "fear", "joy", "sadness", "surprise"]].values
    return texts, labels

# Load train and test data
train_file = "Emotion_data/public_data_test/track_a/train/eng.csv"
test_file = "Emotion_data/public_data_test/track_a/dev/eng.csv"
human_pred = "Emotion_data/eng_test_50 labels.csv"


train_texts, train_labels = load_data(train_file)
test_texts, test_labels = load_data(test_file)
h_texts, h_labels = load_data(human_pred)

# Initialize tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-large")

# Find the max token length dynamically
def find_max_length(texts, tokenizer):
    tokenized_texts = [tokenizer.tokenize(text) for text in texts]
    return max(len(tokens) for tokens in tokenized_texts)

max_length = find_max_length(train_texts + test_texts, tokenizer)  # Find max length from both train & test

print(f"Dynamic max length: {max_length}")

# Tokenize with dynamic max_length
def tokenize_texts(texts, tokenizer, max_length):
    return tokenizer(
        texts,
        max_length=max_length,
        truncation=True,
        padding="max_length",
        return_tensors="pt",
    )

train_encodings = tokenize_texts(train_texts, tokenizer, max_length)
test_encodings = tokenize_texts(test_texts, tokenizer, max_length)
h_encodings = tokenize_texts(h_texts, tokenizer, max_length)


Dynamic max length: 107


In [3]:
class EmotionDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": torch.tensor(self.labels[idx], dtype=torch.float),
        }

# Create datasets
train_dataset = EmotionDataset(train_encodings, train_labels)
test_dataset = EmotionDataset(test_encodings, test_labels)
h_dataset = EmotionDataset(h_encodings, h_labels)


In [4]:
class RobertaClass(torch.nn.Module):
    def __init__(self, num_labels=5):
        super(RobertaClass, self).__init__()
        self.roberta = RobertaModel.from_pretrained("roberta-large")
        self.dropout = torch.nn.Dropout(0.5)

         # Additional fully connected layers
        self.fc1 = torch.nn.Linear(1024, 512)  
        self.fc2 = torch.nn.Linear(512, num_labels)
        #self.classifier = torch.nn.Linear(1024, num_labels)  # 1024 is the output dimension of Roberta

    def forward(self, input_ids, attention_mask):
        # Get the output from Roberta
        output = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        
        # We only care about the [CLS] token output for classification (i.e., first token in the sequence)
        cls_output = output[0][:, 0, :]  # (batch_size, hidden_size) - [CLS] token representation
        
        # Apply dropout for regularization
        cls_output = self.dropout(cls_output)
        
        # Pass through the classifier to get the logits (raw output scores for each class)
        #logits = self.classifier(cls_output)

        # Pass through the first fully connected layer
        x = torch.nn.ReLU()(self.fc1(cls_output))
        
        # Apply dropout after the fully connected layer
        x = self.dropout(x)
        
        # Final output layer for classification
        logits = self.fc2(x)
        
        return logits


In [35]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# Example model initialization
model = RobertaClass(num_labels=5) 
model.to(device)  

optimizer = AdamW(model.parameters(), lr=1e-4)
criterion = torch.nn.BCEWithLogitsLoss()  
epochs = 20

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)

# Example training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(input_ids, attention_mask)
        
        # Compute the loss
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(train_loader)}")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/bhra451f/.conda/envs/SemEval/lib/python3.13/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/20, Loss: 0.5935453068400394
Epoch 2/20, Loss: 0.5838801913027991
Epoch 3/20, Loss: 0.5820670104504837
Epoch 4/20, Loss: 0.5786066715621707
Epoch 5/20, Loss: 0.5783654803227138
Epoch 6/20, Loss: 0.5754801402288365
Epoch 7/20, Loss: 0.5757308346730781
Epoch 8/20, Loss: 0.5734285552463779
Epoch 9/20, Loss: 0.5752320082244501
Epoch 10/20, Loss: 0.5735240757809899


KeyboardInterrupt: 

In [5]:
#Training Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model, optimizer, and criterion initialization
model = RobertaClass(num_labels=5)
model.to(device)
epochs = 20

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)

optimizer = AdamW(model.parameters(), lr=1e-5, eps=1e-8, weight_decay=1e-5)  # Adjusted learning rate (1e-5 decay)
criterion = torch.nn.BCEWithLogitsLoss()

# Scheduler for learning rate decay
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1) #step_size =2

# Data loaders (batch size increased)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

# Early stopping variables
best_loss = float('inf')
epochs = 20

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(input_ids, attention_mask)
        
        # Compute loss
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    scheduler.step()  # Update the learning rate based on the scheduler

    # Print training loss
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(train_loader)}")

    # Save the best model based on loss
    if total_loss < best_loss:
        best_loss = total_loss
        torch.save(model.state_dict(), "best_model.pt")

# Optionally load the best model for evaluation
model.load_state_dict(torch.load("best_model.pt"))


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/bhra451f/.conda/envs/SemEval/lib/python3.13/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/20, Loss: 0.563780050229475
Epoch 2/20, Loss: 0.4126465351595355
Epoch 3/20, Loss: 0.30733570855950687
Epoch 4/20, Loss: 0.22291956417736292
Epoch 5/20, Loss: 0.1971910347666785
Epoch 6/20, Loss: 0.18016777987015903
Epoch 7/20, Loss: 0.16417236544321037
Epoch 8/20, Loss: 0.163989586647966
Epoch 9/20, Loss: 0.16218981126355642
Epoch 10/20, Loss: 0.1597091849904098
Epoch 11/20, Loss: 0.15823694440735386
Epoch 12/20, Loss: 0.1591876811762421
Epoch 13/20, Loss: 0.16054409494720442
Epoch 14/20, Loss: 0.16017440738791675
Epoch 15/20, Loss: 0.1594391815940381
Epoch 16/20, Loss: 0.15979880222921514
Epoch 17/20, Loss: 0.15909522040504853
Epoch 18/20, Loss: 0.15933974045248045
Epoch 19/20, Loss: 0.159943555659383
Epoch 20/20, Loss: 0.1581529397840445


/tmp/ipykernel_722426/237745877.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pt"))


<All keys matched successfully>

In [6]:
# Function to evaluate F1 score on validation set
from transformers import get_linear_schedule_with_warmup

def evaluate(model, val_loader, criterion):
    model.eval()
    val_loss = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            preds = (torch.sigmoid(outputs) > 0.5).float()
            all_preds.append(preds.cpu().numpy())
            all_labels.append(labels.cpu().numpy())

    val_loss /= len(val_loader)
    all_preds = np.vstack(all_preds)
    all_labels = np.vstack(all_labels)
    f1 = f1_score(all_labels, all_preds, average='micro')  # Or use 'macro' for equal weighting

    return val_loss, f1

optimizer = AdamW(model.parameters(), lr=3e-5, eps=1e-8)

total_steps = len(train_loader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0.1 * total_steps, num_training_steps=total_steps)


# Training loop with validation evaluation
best_f1 = 0.0
best_loss = float("inf")

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Gradient clipping
        optimizer.step()

        total_loss += loss.item()

    scheduler.step()  # Update learning rate
    avg_train_loss = total_loss / len(train_loader)

    # **Evaluate on validation set**
    val_loss, val_f1 = evaluate(model, test_loader, criterion)
    
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {val_loss:.4f}, Val F1: {val_f1:.4f}")

    # **Save best model based on F1 score**
    if val_f1 > best_f1:
        best_f1 = val_f1
        torch.save(model.state_dict(), "best_model_f1.pt")

print(f"Best Validation F1 Score: {best_f1:.4f}")


/home/bhra451f/.conda/envs/SemEval/lib/python3.13/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


KeyboardInterrupt: 

In [7]:
# Evaluation
model.eval()
all_preds = []
all_labels = []
test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        
        # Forward pass
        outputs = model(input_ids, attention_mask)
        
        # Apply sigmoid and threshold to get binary predictions
        #preds = (torch.sigmoid(outputs) > 0.5).float()  # Threshold at 0.5 for multi-label
        thresholds = torch.tensor([0.45, 0.5, 0.5, 0.5, 0.5]).to(device)  # Lower threshold for Label 0
        preds = (torch.sigmoid(outputs) > thresholds).float()

        
        # Collect predictions and labels for F1 score
        all_preds.append(preds.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

# Flatten lists and calculate F1 score
all_preds = np.vstack(all_preds)  # Shape: (num_samples, num_labels)
all_labels = np.vstack(all_labels)  # Shape: (num_samples, num_labels)

# Calculate F1 score for each label individually
f1_per_label = f1_score(all_labels, all_preds, average=None)  # F1 score for each label
print("F1 Score per label:")
for idx, score in enumerate(f1_per_label):
    print(f"Label {idx} F1 Score: {score:.4f}")


F1 Score per label:
Label 0 F1 Score: 0.8485
Label 1 F1 Score: 0.7846
Label 2 F1 Score: 0.7200
Label 3 F1 Score: 0.7671
Label 4 F1 Score: 0.6885


In [8]:
# Evaluation with human predictions
model.eval()
all_preds = []
all_labels = []
test_loss = 0.0
correct = 0
total = 0

h_loader = DataLoader(h_dataset, batch_size=8, shuffle=False)

with torch.no_grad():
    for batch in h_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        
        # Forward pass
        outputs = model(input_ids, attention_mask)
        
        # Apply sigmoid and threshold to get binary predictions
        preds = (torch.sigmoid(outputs) > 0.5).float()  # Threshold at 0.5 for multi-label
        
        # Collect predictions and labels for F1 score
        all_preds.append(preds.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

# Flatten lists and calculate F1 score
all_preds = np.vstack(all_preds)  # Shape: (num_samples, num_labels)
all_labels = np.vstack(all_labels)  # Shape: (num_samples, num_labels)

# Calculate F1 score for each label individually
f1_per_label = f1_score(all_labels, all_preds, average=None)  # F1 score for each label
print("F1 Score per label:")
for idx, score in enumerate(f1_per_label):
    print(f"Label {idx} F1 Score: {score:.4f}")


F1 Score per label:
Label 0 F1 Score: 0.4286
Label 1 F1 Score: 0.6122
Label 2 F1 Score: 0.6364
Label 3 F1 Score: 0.6667
Label 4 F1 Score: 0.5714
